<a href="https://colab.research.google.com/github/Sai-sakunthala/Assignment1/blob/main/Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sai-sakunthala (sai-sakunthala-indian-institute-of-technology-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [1]:
from keras.datasets import fashion_mnist
#import wandb
import numpy as np
import math
import random

In [ ]:
wandb.init(
    # set the wandb project where this run will be logged
    project="Fashion-mnist", name = "Images")

wandb: Currently logged in as: sai-sakunthala (sai-sakunthala-indian-institute-of-technology-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [2]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
class_names = ['Tshirt', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'bag', 'Ankleboot']

one_per_label = {}
for image,label in zip(x_train,y_train):
    if label not in one_per_label:
        one_per_label[label] = image

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
for label, image in one_per_label.items():
    wandb.log({"Images": [wandb.Image(image, caption=class_names[label])]})

In [18]:
def input_layer(x):
    x = np.array(x)
    if len(x.shape) == 3:
        a,b,c = x.shape
        x = x.reshape(a,b*c)
    return x

def sigmoid(a_x):
    h_x = np.zeros(len(a_x))
    for i in range(len(a_x)):
        h_x[i] = 1/(1+math.exp(-a_x[i]))
    return h_x

def der_sigmoid(a_x):
    del_sig = np.zeros(len(a_x))
    for i in range(len(a_x)):
        del_sig[i] = a_x[i]*(1-a_x[i])
    return del_sig

def Relu(a_x):
    h_x = np.zeros(len(a_x))
    for i in range(len(a_x)):
        h_x[i] = max(0,a_x[i])
    return h_x

def der_Relu(a_x):
    del_Relu = np.zeros(len(a_x))
    for i in range(len(a_x)):
        if a_x[i] > 0:
            del_Relu[i] = a_x[i]
        else:
            del_Relu[i] = 0
    return del_Relu

def tanh(a_x):
    h_x = np.zeros(len(a_x))
    for i in range(len(a_x)):
        h_x[i] = (math.exp(a_x[i]) - math.exp(-a_x[i]))/(math.exp(a_x[i]) + math.exp(-a_x[i]))
    return h_x

def der_tanh(a_x):
    del_tanh = np.zeros(len(a_x))
    for i in range(len(a_x)):
        del_tanh[i] = 1 - [(math.exp(a_x[i]) - math.exp(-a_x[i]))/(math.exp(a_x[i]) + math.exp(-a_x[i]))]**2
    return del_tanh

def softmax(a_x):
    h_x = np.zeros(len(a_x))
    for i in range(len(a_x)):
        h_x[i] = math.exp(a_x[i])
    h_x = h_x/np.sum(h_x)
    return h_x

def der_softmax(a_x):
    del_softmax = np.zeros(len(a_x))
    for i in range(len(a_x)):
        del_softmax[i] = a_x[i]*(1-a_x[i])

def initialize_weights(num_neurons):
    weights = []
    biases = []
    for i in range(len(num_neurons)-1):
        W = np.random.randn(num_neurons[i+1], num_neurons[i])
        b = np.random.randn(1, num_neurons[i+1])
        weights.append(W)
        biases.append(b)
    return weights, biases

def pre_activation(h_x, W, b):
    a_x = np.dot(W, h_x.T) + b
    a_x = a_x.reshape(len(a_x[0]))
    return a_x

def bce_loss_function(h_x, y):
    idx = np.where(y == 1)
    loss = -np.log(h_x[idx])
    return loss

def mse_loss_function(h_x, y):
    loss = (h_x - y)**2
    return loss

def forward_pass(x, y, weights, biases, activation_func, n_hidden, loss_function):
    activations = []
    pre_activations = []
    for i in range(n_hidden+1):
        if i == 0:
            a_x = pre_activation(x, weights[i], biases[i])
            h_x = activation_func(a_x)
            activations.append(h_x)
            pre_activations.append(a_x)
        elif i == n_hidden:
            a_x = pre_activation(h_x, weights[i], biases[i])
            h_x = softmax(a_x)
            activations.append(h_x)
            pre_activations.append(a_x)
        else:
            a_x = pre_activation(h_x, weights[i], biases[i])
            h_x = activation_func(a_x)
            activations.append(h_x)
            pre_activations.append(a_x)
    loss = loss_function(h_x, y)
    return activations, pre_activations, loss

#a,b,_ = forward_pass(np.array([1,1]), np.array([0]), initialize_weights([2,3,3,2])[0], initialize_weights([2,3,3,2])[1], tanh, 2)

def one_hot_encode(y, num_classes):
    return np.eye(num_classes)[y]

def back_propagation(activations, pre_activations, weights, biases, y, y_hat, n_hidden, activation_deriv, loss_function):
    del_L_a = {}
    del_L_w = {}
    del_L_b = {}
    del_L_h = {}
    for i in range(n_hidden, -1, -1):
        if i == n_hidden:
            if loss_function == bce_loss_function:
                del_L_a[i] = -(y - y_hat)
        del_L_w[i] = np.outer(del_L_a[i], activations[i-1].T)
        del_L_b[i] = del_L_a[i]
        if i == 0:
            break
        del_L_h[i-1] = np.matmul(weights[i].T, del_L_a[i])
        del_L_a[i-1] = np.multiply(del_L_h[i-1], activation_deriv(pre_activations[i-1]))
    return del_L_w, del_L_b

def gradient_descent(dw, db, weights, biases, learning_rate):
    for i in range(len(weights)):
        weights[i] = weights[i] - learning_rate*dw[i]
        biases[i] = biases[i] - learning_rate*db[i]
    return weights, biases

def momentum_gradient(dw, db, weights, biases, learning_rate, beta, prev_u_w, prev_u_b):
    u_w = {}
    u_b = {}
    for i in range(len(weights)):
        if prev_u_w == {} and prev_u_b == {}:
            u_w[i] = learning_rate*dw[i]
            u_b[i] = learning_rate*db[i]
        else:
            u_w[i] = beta*prev_u_w[i] + learning_rate*dw[i]
            u_b[i] = beta*prev_u_b[i] + learning_rate*db[i]
        weights[i] = weights[i] - u_w[i]
        biases[i] = biases[i] - u_b[i]
    prev_u_w = u_w
    prev_u_b = u_b
    return weights, biases, prev_u_w, prev_u_b

def nestrov_gradient(dw, db, weights, biases, learning_rate, beta, prev_u_w, prev_u_b):
    u_w = {}
    u_b = {}
    for i in range(len(weights)):
        if prev_u_w == {} and prev_u_b == {}:
            u_w[i] = learning_rate*dw[i]
            u_b[i] = learning_rate*db[i]
        else:
            u_w[i] = beta*prev_u_w[i] + learning_rate*dw[i]
            u_b[i] = beta*prev_u_b[i] + learning_rate*db[i]
        weights[i] = weights[i] - u_w[i]
        biases[i] = biases[i] - u_b[i]
    prev_u_w = u_w
    prev_u_b = u_b
    return weights, biases, prev_u_w, prev_u_b

def Neuralnet(x_train, y_train, n_hidden, n_neurons_hidden, epochs, batch_size, activation, optimization, learning_rate, weight_decay, loss_function):
    x_train = input_layer(x_train)
    y_train = one_hot_encode(y_train, len(np.unique(y_train)))
    features = x_train.shape[1]
    classes = len(np.unique(y_train))
    num_neurons = [features] + [n_neurons_hidden]*(n_hidden) + [classes]
    weights, biases = initialize_weights(num_neurons)
    activation_func = {"sigmoid": sigmoid, "tanh": tanh, "relu": Relu}[activation]
    activation_deriv = {"sigmoid": der_sigmoid, "tanh": der_tanh, "relu": der_Relu}[activation]
    optimization_func = {"momentum": momentum_gradient, "sgd": gradient_descent, "nestrov": nestrov_gradient}[optimization]
    loss_function = {"bce": bce_loss_function, 'mse': mse_loss_function}[loss_function]
    #epochs
    if optimization_func == gradient_descent:
        for epoch in range(epochs):
            for i in range(0, len(x_train), batch_size):
                x_batch = x_train[i:i + batch_size]
                y_batch = y_train[i:i + batch_size]
                dw = {}
                db = {}
                for x,y in zip(x_batch,y_batch):
                    activations, pre_activations, loss = forward_pass(x, y, weights, biases, activation_func, n_hidden, loss_function)
                    del_L_w, del_L_b = back_propagation(activations, pre_activations, weights, biases, y, activations[-1], n_hidden, activation_deriv, loss_function)
                    for key,value in del_L_w.items():
                        if key not in dw:
                            dw[key] = value
                        else:
                            dw[key] = dw[key] + value
                    for key,value in del_L_b.items():
                        if key not in db:
                            db[key] = value
                        else:
                            db[key] = db[key] + value
                weights, biases = optimization_func(dw, db, weights, biases, learning_rate)
    elif optimization_func == momentum_gradient:
        prev_u_w = {}
        prev_u_b = {}
        for epoch in range(epochs):
            for i in range(0, len(x_train), batch_size):
                x_batch = x_train[i:i + batch_size]
                y_batch = y_train[i:i + batch_size]
                dw = {}
                db = {}
                for x,y in zip(x_batch,y_batch):
                    activations, pre_activations, loss = forward_pass(x, y, weights, biases, activation_func, n_hidden, loss_function)
                    del_L_w, del_L_b = back_propagation(activations, pre_activations, weights, biases, y, activations[-1], n_hidden, activation_deriv, loss_function)
                    for key,value in del_L_w.items():
                        if key not in dw:
                            dw[key] = value
                        else:
                            dw[key] = dw[key] + value
                    for key,value in del_L_b.items():
                        if key not in db:
                            db[key] = value
                        else:
                            db[key] = db[key] + value
                weights, biases, prev_u_w, prev_u_b = optimization_func(dw, db, weights, biases, learning_rate, weight_decay, prev_u_w, prev_u_b)
    elif optimization_func == nestrov_gradient:
        prev_u_w = {}
        prev_u_b = {}
        for epoch in range(epochs):
            for i in range(0, len(x_train), batch_size):
                x_batch = x_train[i:i + batch_size]
                y_batch = y_train[i:i + batch_size]
                dw = {}
                db = {}
                for x,y in zip(x_batch,y_batch):
                    activations, pre_activations, loss = forward_pass(x, y, weights, biases, activation_func, n_hidden, loss_function)
                    if i == 0:
                        del_L_w, del_L_b = back_propagation(activations, pre_activations, weights, biases, y, activations[-1], n_hidden, activation_deriv, loss_function)
                    else:
                        look_ahead_weights = {key: weights[key] - weight_decay*prev_u_w[key] for key in range(len(weights))}
                        look_ahead_biases = {key: biases[key] - weight_decay*prev_u_b[key] for key in range(len(weights))}
                        del_L_w, del_L_b = back_propagation(activations, pre_activations, look_ahead_weights, look_ahead_biases, y, activations[-1], n_hidden, activation_deriv, loss_function)
                    for key,value in del_L_w.items():
                        if key not in dw:
                            dw[key] = value
                        else:
                            dw[key] = dw[key] + value
                    for key,value in del_L_b.items():
                        if key not in db:
                            db[key] = value
                        else:
                            db[key] = db[key] + value
                weights, biases, prev_u_w, prev_u_b = optimization_func(dw, db, weights, biases, learning_rate, weight_decay, prev_u_w, prev_u_b)

Neuralnet(np.array([[1,1],[0,0],[1,1],[2,2]]), np.array([0,1,0,1]), 2, 3, 1, 2, 'sigmoid', 'momentum', 0.01, 0.9, 'bce')